In [1]:
# An unstrustured 2D circular control volume mesh generator for airofil geometries.
# Yigit Saygili - ENSAM PARISTECH 2024

# 1- Importing the Packages
import Gmsh: gmsh
using LinearAlgebra
using Plots

# 2- Defining a Function for Generating the Coordinates of a Circle
function circle_coordinates(center::Tuple{Float64, Float64}, radius::Float64, num_points::Int)
    points = []
    for i in 1:num_points
        angle = 2 * π * (i - 1) / num_points
        x = center[1] + radius * cos(angle)
        y = center[2] + radius * sin(angle)
        push!(points, (x, y))
    end
    return points
end

points = circle_coordinates((0.0,0.0), 1.0, 100)

push!(points, points[1])
x_points = [p[1] for p in points]
y_points = [p[2] for p in points]
plot(x_points, y_points, label= "Outer Boundary", xlabel="x/c", ylabel="y/c", title="Circle", ratio=1)


# 3- Defining a Function for Generating the Coordinates of an Airfoil
function naca_4digit(m, p, t, n_points)
    p /= 100
    m /= 100
    t /= 100
    c = 1.0
    x = LinRange(0, 1, n_points)

    y_c = zeros(n_points)
    for i in 1:n_points
        if x[i] < p
            y_c[i] = (m / p^2) * (2 * p * x[i] - x[i]^2)
        else
            y_c[i] = (m / (1 - p)^2) * ((1 - 2 * p) + 2 * p * x[i] - x[i]^2)
        end
    end

    y_t = 5 * t * c * (0.2969 * sqrt.(x) .- 0.1260 * x .- 0.3516 .* x.^2 .+ 0.2843 .* x.^3 .- 0.1015 .* x.^4)
    y_u = y_c .+ y_t
    y_l = y_c .- y_t

    x_coords = vcat(x, reverse(x))
    y_coords = vcat(y_u, reverse(y_l))

    combined_coords = Tuple{Float64, Float64}[]
    for i in 1:n_points * 2
        push!(combined_coords, (x_coords[i], y_coords[i]))
    end

    return combined_coords
end

function plot_airfoil(m, p, t, n)
    coords = naca_4digit(m, p, t, n)
    x_values = [coord[1] for coord in coords]
    y_values = [coord[2] for coord in coords]
    plot(x_values, y_values, label="Inner Boundary", xlabel="x/c", ylabel="y/c", title="4-digit NACA Airfoil", ratio = 1)
end

# Example: NACA 2412
plot_airfoil(2, 4, 12, 100)


# 4- Defining a Function for Generating the Control Volume Mesh
function generate_mesh(outer_vertices::Vector{}, inner_vertices::Vector{}, lc_outer::Float64, lc_inner::Float64)
    # 1/7: Initialize Gmsh
    gmsh.initialize()

    # 2/7: Create a new model and set terminal output
    gmsh.option.setNumber("General.Terminal", 1)
    gmsh.model.add("donut_shape_circles")

    # 3/7: Generate points for the outer and inner circles
    outer_points = Int[]
    inner_points = Int[]

    # Define outer circle points with local mesh resolution
    for i in 1:length(outer_vertices)
        x, y = outer_vertices[i]
        # Assign outer mesh resolution to each point in the outer circle
        point_label = gmsh.model.geo.addPoint(x, y, 0, lc_outer, i)
        push!(outer_points, point_label)
    end

    # Define inner circle points with local mesh resolution
    for i in 1:length(inner_vertices)
        x, y = inner_vertices[i]
        # Assign inner mesh resolution to each point in the inner circle (hole)
        point_label = gmsh.model.geo.addPoint(x, y, 0, lc_inner, i + length(outer_vertices))
        push!(inner_points, point_label)
    end

    # 4/7: Define lines (edges) for the outer and inner circles
    outer_lines = Int[]
    inner_lines = Int[]

    # Create the outer boundary lines (connect consecutive points on the outer circle)
    for i in 1:length(outer_points)
        start_point = outer_points[i]
        end_point = outer_points[mod(i, length(outer_points)) + 1]
        push!(outer_lines, gmsh.model.geo.addLine(start_point, end_point, i))
    end

    # Create the inner boundary lines (connect consecutive points on the inner circle)
    for i in 1:length(inner_points)
        start_point = inner_points[i]
        end_point = inner_points[mod(i, length(inner_points)) + 1]
        push!(inner_lines, gmsh.model.geo.addLine(start_point, end_point, i + length(outer_points)))
    end

# 5/7: Define two curve loops (one for the outer boundary and one for the inner hole)
outer_loop = gmsh.model.geo.addCurveLoop(outer_lines, 1)
inner_loop = gmsh.model.geo.addCurveLoop(inner_lines, 2)

# 6/7: Define the surface (donut shape with hole)
gmsh.model.geo.addPlaneSurface([1, 2], 1)

# 7/7: Synchronize geometry with Gmsh
gmsh.model.geo.synchronize()

# 8/7: Generate mesh
gmsh.model.mesh.generate(2)

# 9/7: Write mesh to file (optional)
gmsh.write("data/donut_shape_circles.msh")

# 10/7: Visualize the mesh using Gmsh GUI (optional)
gmsh.fltk.run()

# 11/7: Finalize Gmsh
gmsh.finalize()
end

function plot_control_volume(m, p, t, ni, cx, cy, r, no)
    coords = naca_4digit(m, p, t, ni)
    x_values = [coord[1] for coord in coords]
    y_values = [coord[2] for coord in coords]

    # Generate circle coordinates
    points = circle_coordinates((cx, cy), r ,no)
    
    # To close the circle, add the first point to the end
    push!(points, points[1])

    # Extract x and y coordinates of the circle
    x_points = [p[1] for p in points]
    y_points = [p[2] for p in points]

    # Plot the circle first, then overlay the NACA airfoil
    plot(x_points, y_points, label="Outer Boundary", xlabel="x/c", ylabel="y/c", title="Control Volume", ratio=1, linewidth=2)
    plot!(x_values, y_values, label="Inner Boundary", linewidth=2)
end

plot_control_volume(2, 4, 12, 50, 0.5, 0.0, 1.5, 50)


# 5- Obtaining the Coordinates of Inner and Outer Boundaries
outer_vertices = circle_coordinates((0.0,0.0), 2.0, 20)
inner_vertices = naca_4digit(2, 4, 12, 10)


# 6- Generating the Mesh Inside the Control Volume
generate_mesh(outer_vertices, inner_vertices, 0.1, 0.01)


# 7- Refining the Mesh
control_volume_center_x = 0.5
control_volume_center_y = 0.0
control_volume_radius = 3.0
outer_boundary_panel_number = 50

airfoil_camber = 2
airfoil_camber_location = 4
airfoil_thickness = 12
inner_boundary_panel_number = 50

outer_boundary_mesh_size = 0.5
inner_boundary_mesh_size = 0.1

outer_vertices = circle_coordinates((control_volume_center_x, control_volume_center_y), control_volume_radius, outer_boundary_panel_number)
inner_vertices = naca_4digit(airfoil_camber, airfoil_camber_location, airfoil_thickness, inner_boundary_panel_number)
generate_mesh(outer_vertices, inner_vertices, outer_boundary_mesh_size, inner_boundary_mesh_size)


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 20%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 30%] Meshing curve 9 (Line)
Info    : [ 30%] Meshing curve 10 (Line)
Info    : [ 30%] Meshing curve 11 (Line)
Info    : [ 30%] Meshing curve 12 (Line)
Info    : [ 40%] Meshing curve 13 (Line)
Info    : [ 40%] Meshing curve 14 (Line)
Info    : [ 40%] Meshing curve 15 (Line)
Info    : [ 40%] Meshing curve 16 (Line)
Info    : [ 50%] Meshing curve 17 (Line)
Info    : [ 50%] Meshing curve 18 (Line)
Info    : [ 50%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 60%] Meshing curve 21 (Line)
Info    : [ 60%] Meshing curve 22 (Line)
Info    : [ 60%] Meshing curve 23 (Line)
Info    : [ 60%] Meshing curve 24 (Line)
I